In [ ]:
import subprocess
import pickle
import pandas as pd
import nltk
import utils
from utils import *

SENT_COL="sent"
TOKEN_COL="token"
SHAPE_COL="token_shape"
POS_COL="pos"
NER_COL="ner"

filename="crf_fr.pickle"

file = open(filename, 'rb')
crf = pickle.load(file)
file.close()

print("Loaded: "+filename)

sent = ""
while True:
    print()
    sent = input("Give a sentence in French (q to exit): ")
    if sent in ['q']:
        break

    file1= open("fr.tmp","w")
    file1.write(sent)
    file1.flush()
    file1.close

    import os, sys
    import time

    def getSize(filename):
        if os.path.isfile(filename): 
            st = os.stat(filename)
            return st.st_size
        else:
            return -1

    def wait_download(file_path):
        current_size = getSize(file_path)
        print("File size")
        i = 0
        while current_size !=getSize(file_path) or getSize(file_path)==0 or i > 10:
            current_size =getSize(file_path)
            print("current_size:"+str(current_size))
            time.sleep(1)# wait download
            i = i + 1

    wait_download("fr.tmp")
    
    print("File Written")

    batcmd = "fr-tag/cmd/tree-tagger-french"

    result = subprocess.check_output([batcmd, 'fr.tmp'])
    
    words = []
    posTags = []
    i=0
    for word in result.split():
        word=word.decode("utf-8")
        if i % 3 == 0:
            words.append(word)
        if i % 3 == 1:
            if len(word.split(":"))>1:
                posTags.append(word.split(':')[0])
            else:
                posTags.append(word)
        i = i + 1

    data=[]
    for i in range(len(words)):
        word = words[i]
        pos = posTags[i]
        ner = "unknown"
        data.append([1,word,pos,ner])
    df = pd.DataFrame(data, columns = [ SENT_COL, TOKEN_COL, POS_COL, NER_COL ]) 


    analyseTokens(df)

    getter_sent = SentenceGetter(df)
    pred_sentence = getter_sent.sentences

    X_sent = [sent2features(s) for s in pred_sentence]
    y_pred = crf.predict(X_sent)[0]

    df[NER_COL]=y_pred
    df = df.drop([SENT_COL,SHAPE_COL], axis=1)

    df.rename(columns={TOKEN_COL:TOKEN_COL.upper(),POS_COL:POS_COL.upper(),NER_COL:NER_COL.upper()}, 
                     inplace=True)
    print("\n")
    print(df)
print("\nWhat would it mean if you got to quit?")

Loaded: crf_fr.pickle

Give a sentence in French (q to exit): Je m'appelle Ken et je viens de Paris.
File size
File Written


     TOKEN   POS  NER
0       Je   PRO    O
1       m'   PRO    O
2  appelle   VER    O
3      Ken   NAM  LOC
4       et   KON    O
5       je   PRO    O
6    viens   VER    O
7       de   PRP    O
8    Paris   NAM  LOC
9        .  SENT    O

Give a sentence in French (q to exit): Je m'appelle Ken et je viens de Paris.
File size
File Written


     TOKEN   POS  NER
0       Je   PRO    O
1       m'   PRO    O
2  appelle   VER    O
3      Ken   NAM  LOC
4       et   KON    O
5       je   PRO    O
6    viens   VER    O
7       de   PRP    O
8    Paris   NAM  LOC
9        .  SENT    O

Give a sentence in French (q to exit): Je m'appelle Ken et je viens de Pari.
File size
File Written


     TOKEN   POS  NER
0       Je   PRO    O
1       m'   PRO    O
2  appelle   VER    O
3      Ken   NAM  LOC
4       et   KON    O
5       je   PRO    O
6    viens   VER    O
7      